In [1]:
import pandas as pd
import numpy as np
import joblib
import os

In [11]:
import os
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm

pd.set_option("display.precision", 4)

DATA_PATH = os.path.join("..", "data", "imputed_1min")
GROUPED_BY_DATE_PATH = os.path.join("..", "data", "grouped_by_date")

FILES = os.listdir(DATA_PATH)
FILES_GROUPED_BY_DATE = os.listdir(GROUPED_BY_DATE_PATH)


# noinspection PyTypeChecker
def price_based_features_for_day(daily_data, offset=30, previous_close=0):
    close = daily_data["close"].to_numpy()  # daily closing prices as np array
    high = daily_data["high"].to_numpy()  # daily high prices as np array
    low = daily_data["low"].to_numpy()  # daily low prices as np array

    # initialize output with first N rows
    out = {
        "keep_row": [False] * offset,  # these rows will be deleted later, first N minutes of a day
        "last_close": [0] + list(close[:offset - 1]),
        "second_last_close": [0, 0] + list(close[:offset - 2]),
        "last_high": [0] + list(high[:offset - 1]),
        "second_last_high": [0, 0] + list(high[:offset - 2]),
        "last_low": [0] + list(low[:offset - 1]),
        "second_last_low": [0, 0] + list(low[:offset - 2]),
        "deviation_from_5_min_avg": [0] * offset,
        "deviation_from_10_min_avg": [0] * offset,
        "deviation_from_15_min_avg": [0] * offset,
        "deviation_from_30_min_avg": [0] * offset,
        "last_5_min_high": [0] * offset,
        "last_10_min_high": [0] * offset,
        "last_30_min_high": [0] * offset,
        "last_5_min_low": [0] * offset,
        "last_10_min_low": [0] * offset,
        "last_30_min_low": [0] * offset,
        "last_5_min_interval": [0] * offset,
        "last_10_min_interval": [0] * offset,
        "last_30_min_interval": [0] * offset,
        "daily_avg_until_now": [0] * offset,
        "deviation_from_daily_avg_until_now": [0] * offset,
    }

    for i in range(offset, len(close)):
        # Dont include the current OHCL values to the calculation
        # Use values until 1 min before
        subset_close = close[i-30:i]
        subset_high = high[i-30:i]
        subset_low = low[i-30:i]

        out["keep_row"].append(True)

        out["last_close"].append(subset_close[-1])  # last observed closing price value
        out["second_last_close"].append(subset_close[-2])  # second last observed closing price value

        out["last_high"].append(subset_close[-1])  # last observed closing price value
        out["second_last_high"].append(subset_close[-2])  # second last observed closing price value

        out["last_low"].append(subset_close[-1])  # last observed closing price value
        out["second_last_low"].append(subset_close[-2])  # second last observed closing price value

        # deviation of the last observed closing price from averages
        out["deviation_from_5_min_avg"].append(subset_close[-1] - np.mean(subset_close[-5:]))
        out["deviation_from_10_min_avg"].append(subset_close[-1] - np.mean(subset_close[-10:]))
        out["deviation_from_15_min_avg"].append(subset_close[-1] - np.mean(subset_close[-15:]))
        out["deviation_from_30_min_avg"].append(subset_close[-1] - np.mean(subset_close))

        # highest value of last 5 minutes
        out["last_5_min_high"].append(np.max(subset_high[-5:]))
        out["last_10_min_high"].append(np.max(subset_high[-10:]))
        out["last_30_min_high"].append(np.max(subset_high[-30:]))

        # lowest value of last 5 minutes
        out["last_5_min_low"].append(np.min(subset_low[-5:]))
        out["last_10_min_low"].append(np.min(subset_low[-10:]))
        out["last_30_min_low"].append(np.min(subset_low[-30:]))

        # daily average closing price until now
        out["daily_avg_until_now"].append(np.mean(close[:i]))
        out["deviation_from_daily_avg_until_now"].append(subset_close[-1] - np.mean(close[:i]))

    out["last_5_min_interval"] = np.array(out["last_5_min_high"]) - np.array(out["last_5_min_low"])
    out["last_10_min_interval"] = np.array(out["last_10_min_high"]) - np.array(out["last_10_min_low"])
    out["last_30_min_interval"] = np.array(out["last_30_min_high"]) - np.array(out["last_30_min_low"])

    out["previous_day_close"] = [previous_close] * len(close)

    return pd.DataFrame(data=out)


def get_concatenated_price_based_features(grouped_data):
    price_based_features = pd.DataFrame()
    prev_close = None

    for i in tqdm(grouped_data.values()):
        if len(price_based_features) == 0:
            price_based_features = price_based_features_for_day(i)
            prev_close = price_based_features["last_close"].iloc[-1]
        else:
            todays_pbf = price_based_features_for_day(i, previous_close=prev_close)
            price_based_features = pd.concat([price_based_features, todays_pbf], axis=0)
    price_based_features.reset_index(inplace=True)
    return price_based_features


def apply_price_based_feature_generator(label):
    # read grouped data
    try:
        grouped_data = joblib.load(os.path.join(GROUPED_BY_DATE_PATH, f"{label}.joblib"))
    except FileNotFoundError:
        raise Exception("This label is not included in our dataset.")

    price_based_features = get_concatenated_price_based_features(grouped_data)

    return price_based_features


def extract_price_based_feature_for_stock(label):
    features = apply_price_based_feature_generator(label)
    features = features.rename(columns={"index": "intraday_index"})

    return features


In [12]:
features = extract_price_based_feature_for_stock("AKBNK")

100%|██████████████████████████████████████████████████████████████████████████████| 1002/1002 [00:46<00:00, 21.49it/s]


In [15]:
features.iloc[410:430]

,intraday_index,keep_row,last_close,second_last_close,last_high,second_last_high,last_low,second_last_low,deviation_from_5_min_avg,deviation_from_10_min_avg,...,last_30_min_high,last_5_min_low,last_10_min_low,last_30_min_low,last_5_min_interval,last_10_min_interval,last_30_min_interval,daily_avg_until_now,deviation_from_daily_avg_until_now,previous_day_close
410,410,True,6.2353,6.2353,6.2353,6.2353,6.2353,6.2353,0.0000,0.0000,...,6.2433,6.2353,6.2353,6.2353,0.000,0.008,0.008,6.2435,-0.0082,0.0000
411,411,True,6.2353,6.2353,6.2353,6.2353,6.2353,6.2353,0.0000,0.0000,...,6.2433,6.2353,6.2353,6.2353,0.008,0.008,0.008,6.2435,-0.0082,0.0000
412,412,True,6.2353,6.2353,6.2353,6.2353,6.2353,6.2353,0.0000,0.0000,...,6.2433,6.2353,6.2353,6.2353,0.008,0.008,0.008,6.2435,-0.0082,0.0000
413,413,True,6.2353,6.2353,6.2353,6.2353,6.2353,6.2353,0.0000,0.0000,...,6.2433,6.2353,6.2353,6.2353,0.008,0.008,0.008,6.2434,-0.0081,0.0000
414,414,True,6.2433,6.2353,6.2433,6.2353,6.2433,6.2353,0.0064,0.0072,...,6.2433,6.2353,6.2353,6.2353,0.008,0.008,0.008,6.2434,-0.0001,0.0000
415,415,True,6.2353,6.2433,6.2353,6.2433,6.2353,6.2433,-0.0016,-0.0008,...,6.2433,6.2353,6.2353,6.2353,0.008,0.008,0.008,6.2434,-0.0081,0.0000
416,416,True,6.2353,6.2353,6.2353,6.2353,6.2353,6.2353,-0.0016,-0.0008,...,6.2433,6.2353,6.2353,6.2353,0.008,0.008,0.008,6.2434,-0.0081,0.0000
417,417,True,6.2353,6.2353,6.2353,6.2353,6.2353,6.2353,-0.0016,-0.0008,...,6.2433,6.2353,6.2353,6.2353,0.008,0.008,0.008,6.2434,-0.0081,0.0000
418,418,True,6.2353,6.2353,6.2353,6.2353,6.2353,6.2353,-0.0016,-0.0008,...,6.2433,6.2353,6.2353,6.2353,0.008,0.008,0.008,6.2434,-0.0081,0.0000
419,419,True,6.2353,6.2353,6.2353,6.2353,6.2353,6.2353,0.0000,-0.0008,...,6.2433,6.2353,6.2353,6.2353,0.008,0.008,0.008,6.2434,-0.0080,0.0000
